## Roomba simulation

* Room that is 15 x 20 ft.
each"tick' (unit of time) the roomba moves 1 foot untuill it hits a wall. Whatever spot(1 ft x 1 ft square) the roomba is in at the  time considered cleanred.
* The roomba does not keep track of where its been the only thing is ithe number of degress it las turned.
* Oure defalut foomba follows these rules:
1. move in a straight line until it hits a wall
2. Rotate randomly from 90 to 270 degrees


In [15]:
import math
import random
import matplotlib.pyplot as plt
import statistics as st

In [16]:
%matplotlib inline

In [17]:
def angle_to_x_y(degrees, dist=1):
    radians = math.radians(degrees)
    x = dist * math.cos(radians)
    y = dist * math.sin(radians)
    return x, y

In [18]:
angle_to_x_y(90, 1)

(6.123233995736766e-17, 1.0)

In [19]:
class Room:
    """
    Responsibilities:
    -Dimiensions (width * height)
    keep track of how much of the room hsas been cleaned
    """
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.clean_squares = set()
        
    def reset(self):
        self.clean_squares = set()
        
    def pick_random_location(self):
        x = random.uniform(0, self.width)
        y = random.uniform(0, self.height)
        return x, y
    
    def how_much_cleaned(self):
        return len(self.clean_squares) / self.total_squares #(self.width * self.height)
    
    def total_squares(self):
        return self.width * self.height
    
    def in_room(self, x, y):
        if 0 < x < self.width and 0 < y < self.height:
            return True       # now it is in the room
        else:
            return False
        
    def clean_square(self, x, y):
        x = int(x)
        y = int(y)
        self.clean_squares.add((x, y))
    

In [20]:
bedroom = Room(4, 3)
print(bedroom.how_much_cleaned())
bedroom.clean_square(1.8, 0.9)
print(bedroom.how_much_cleaned())


TypeError: unsupported operand type(s) for /: 'int' and 'method'

In [ ]:
class Roomba:
    """
    Responsibilities:
    - know it currentangle
    - have a response for hitting a wall
    - report its relative x and y movement
    - know its speed
    - change its angle
    """
    
    def __init__(delf, spleed=1, angle=0):
        self.angle = angle
        self.speed = speed
        
    def move(self):
        return angle_to_x_y(self.angle, self.speed)
    
    def turn(self, turn_amount):
        self.angle += turn_amount
        self.angle %= 360
        
    def hit_wall(self):
        self.turn(random.uniform(90, 270))
        

In [ ]:
class Simulation:
    """
    Responsibilities:
    - place the Roomba
    - figure out where ther Roomba is moving (by asking it)
    - updating the room
    - iterating over ticks
    - keeping track of some stats as the room is cleaned
    
    Collaborators:
    - Roomba
    - Room
    """
    
    def __init__(slef, room, roomba):
        self.room =room
        self.roomba = roomba
        
        self.ticks = 0
        
        initial_x, initial_y = self.room.pick_random_locations()
        initial_angle = random.uniform(0, 360)
        
        self.current_x = initial_x
        self.current_y = initial_y
        self.current_angle = initial_angle
        
    def update_roomba(self):
        change_x, change_y = self.roomba.move()
        
        #check to see if we've hit the wall
        #if so, let teh rromba know and get new values
        
        new_x = self.current_x + change_x
        new_y = self.current_y + change_y
        
        if self.room.in_room(new_x, new_y):
            # the roomba moves!
            self.current_x = new_x
            self.current_y = new_y
        else:
            self.roomba.hit_wall()
            
    def iterate(self):
        self.ticks += 1
        self.update_roomba()
        self.room.clean_square(self.current_x, self.current_y)
        
    def get_stats(self):
        return {
            'ticks': self.ticks,
            'ratio_cleaned' : self.room.how_much_cleaned(),
            'roomba_location': self.current_x, self.current_y),
            'roomba_angle': self.roomba.angle
        }
        
    def run_simulation(self):
        self.reset()
        self.room.clean_square(self.current_x, self.current_y)
        while self.room.how_much_cleaned() < 1.0:
            self.iterate()
        
        print("ROOM CLEAN!!!")
        print(self.get_stats())
        
        

In [ ]:
sally = Roomba()
bedroom = Room(4,3)

#sim1 = Simulation(bedroom, sally)
#results = sim1.run_simulation()


In [ ]:
'''
for _ in range(20):
    sim1
'''